In [1]:
import os
import csv
import sys
import copy
import logging
import numpy as np
#from stats import *
import pandas as pd
import seaborn as sns
from scipy import stats
#from Layer import Layer
import tensorflow as tf 
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.model_selection import train_test_split, KFold
from sklearn.datasets import fetch_california_housing, make_regression
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
def write_stats(samples, features, loss, epochs, lr, batch_size, cross_k, r2, mse_train, mae_train, residual_train):
    with open('../data/stats.csv', 'a') as f:
        newrow = [samples, features, loss, epochs, lr, batch_size, cross_k, r2, mse_train, mae_train, residual_train]
        writer = csv.writer(f)
        writer.writerow(newrow)

def fill_dataset(dataframe: DataFrame):
    for column in dataframe:
        if dataframe[column].dtype != 'object':
            dataframe[column] = dataframe[column].fillna(dataframe[column].mean())
    return dataframe

def normalize_dataset(X):
    return tf.keras.utils.normalize(X)

def remove_outliers(X, threshold=7):
    z = np.abs(stats.zscore(X))
    return X[(z<threshold).all(axis=1)][:, 0:-1], X[(z<threshold).all(axis=1)][: ,-1]

def make_dataset(X_data,y_data,k):
    X_data, y_data = remove_outliers(np.concatenate([X_data, y_data], axis=1))
    def gen():
        for train_index, test_index in KFold(k).split(X_data):
            X_train, X_test = X_data[train_index], X_data[test_index]
            XN_train, XN_test = normalize_dataset(X_data[train_index]), normalize_dataset(X_data[test_index])
            y_train, y_test = y_data[train_index], y_data[test_index]
            yield X_train,XN_train,y_train,X_test,XN_test,y_test

    return tf.data.Dataset.from_generator(gen, (tf.double,tf.double,tf.double,tf.double,tf.double,tf.double))


In [3]:
total_columns=["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  
k = 2
epochs = 3000
lr = 0.01
batch_size = 120

#XR, yR = make_regression(n_samples=1000, n_features=2, n_informative=5, noise=50, random_state=5)
#X_train, X_test, y_train, y_test = train_test_split(XR, yR, test_size=0.20, random_state=2)
#columns = ["{0}".format(total_columns[i]) for i in range(XR.shape[1]+1)]
#dataframe = pd.DataFrame(np.concatenate([XR, np.reshape(yR, [-1, 1])], axis=1), columns=columns)

stocks = pd.read_csv("../data/datasets/all_stocks_5yr.csv")
dataframe = pd.DataFrame(stocks)
filter = dataframe["Name"]=="AAPL"
dataframe = dataframe.where(filter).dropna()
data = pd.DataFrame()

data = dataframe.drop('Name', axis=1)
data['target'] = dataframe['open'].fillna(dataframe['open'].mean())-dataframe['close'].fillna(dataframe['close'].mean())
#data['type'] = 1 if dataframe['open']-dataframe['close']>=0 else 0
data.head()

,date,open,high,low,close,volume,target
1259,2013-02-08,67.7142,68.4014,66.8928,67.8542,158168416.0,-0.1400
1260,2013-02-11,68.0714,69.2771,67.6071,68.5614,129029425.0,-0.4900
1261,2013-02-12,68.5014,68.9114,66.8205,66.8428,151829363.0,1.6586
1262,2013-02-13,66.7442,67.6628,66.1742,66.7156,118721995.0,0.0286
1263,2013-02-14,66.3599,67.3771,66.2885,66.6556,88809154.0,-0.2957


In [4]:
vif_data = pd.DataFrame()
vif_data["features"] = data.drop('date', axis=1).columns
vif_data["VIF"] = [variance_inflation_factor(data.drop('date', axis=1).values, i) for i in range(len(data.drop('date', axis=1).columns))]
print(vif_data)

  features           VIF
0     open           inf
1     high  5.060050e+04
2      low  4.511802e+04
3    close           inf
4   volume  2.630125e+00
5   target           inf


C:\Users\Utente\AppData\Roaming\Python\Python39\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\Utente\AppData\Roaming\Python\Python39\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\Utente\AppData\Roaming\Python\Python39\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [5]:
class Layer(tf.Module):
    def __init__(self, activation=tf.identity):
        self.activation = activation
        self.build = False
        self.normalized = False
        
    @tf.function(reduce_retracing=True)
    def xavier_init(self, shape):
        in_dim, out_dim = shape
        xavier_lim = tf.sqrt(6.)/tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
        weight_vals = tf.cast(tf.random.uniform(shape=(in_dim, out_dim), 
                                        minval=-xavier_lim, maxval=xavier_lim, seed=22, dtype=tf.float32), dtype=tf.double)
        #xavier_lim = tf.sqrt(6.)/tf.sqrt(tf.cast(shape, tf.float32))
        #weight_vals = tf.cast(tf.random.uniform(shape=(shape,), 
        #                                minval=-xavier_lim, maxval=xavier_lim, seed=22, dtype=tf.float32), dtype=tf.double)
        
        return weight_vals

    @tf.function
    def __call__(self, X):
        if not self.build:
            n_samples, n_features = X.shape
            self.weights = tf.Variable(self.xavier_init(shape=(n_samples, n_features)), name="Weights", dtype=tf.double, trainable=True, )

            self.bias = tf.Variable(tf.ones(shape=1, dtype=tf.double), name="Bias", dtype=tf.double, trainable=True )
            self.build = True

        z = tf.add(tf.multiply(X, self.weights), self.bias)

        #print(X.shape,"*",self.weights.shape,"+",self.bias.shape, "=", z.shape)
        return self.activation(z), self.bias


In [6]:
class CiclicAI(tf.Module):
    def __init__(self, epochs=100, lr=0.01, batch_size=50):
        self.epochs = epochs
        self.batch_counter = 0
        self.batch_size = batch_size
        self.optimizer = tf.keras.optimizers.Adam(lr=lr)
        self.compiled = False
        self.loss_history = [e for e in range(epochs)]
        self.r2_history = [e for e in range(epochs)]
    
    def verify_batch(self, X):
        if self.batch_counter >= X:
            self.batch_counter = 0

    def calc_metrics(self, e, loss, y_train, predicted_train):
        self.loss_history[e] = loss
        self.r2_history[e] = self.r2(y_train, predicted_train)

    def r2(self, y, y_pred):
        return tf.subtract(
            tf.convert_to_tensor(1, dtype=tf.float64), 
            tf.divide(
                tf.reduce_sum(tf.square(tf.subtract(y, y_pred))),
                tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y_pred))))
            )
        )
    
    @tf.function
    def lasso(self, weights):
        return tf.reduce_sum(tf.norm(weights))

    @tf.function
    def ridge(self, weights):
        return tf.reduce_sum(tf.square(tf.norm(weights)))

    @tf.function(reduce_retracing=True)
    def loss(self, y, predicted, weights):
        return tf.add(tf.add(tf.losses.MSE(y, predicted), self.lasso(weights)), self.ridge(weights))

    @tf.function(reduce_retracing=True)
    def compile(self):
        if not self.compiled:
            self.layer_reg1 = Layer(tf.nn.relu)
            self.compiled = True

    @tf.function(reduce_retracing=True)
    def _predict(self, X, coef, bias):
        return tf.reduce_mean(tf.add(tf.multiply(X, coef), bias), axis=1)

    def predict_test(self, X_test):
        self.predicted_test = self._predict(X_test, self.coef, self.bias)

    def predict(self, x):
        return self._predict(x, self.coef, self.bias)
    
    @tf.function(reduce_retracing=True)
    def _forward(self, X_train):
        return self.layer_reg1(X_train)

    @tf.function(reduce_retracing=True)
    def _backprop(self, weights, X_train, y_train):
        weights, bias = self._forward(weights)
        coef = tf.reduce_mean(tf.add(tf.matmul(tf.transpose(X_train[self.batch_counter:self.batch_counter+self.batch_size]), weights), bias), axis=0)
        predicted = self._predict(X_train, coef, bias)

        return self.loss(y_train, predicted, weights), coef, bias, predicted

    def fit(self, X_train, y_train):
        self.verify_batch(X_train.shape[0])
        weights = normalize_dataset(X_train[self.batch_counter:self.batch_counter+self.batch_size])
        for e in range(self.epochs):
            with tf.GradientTape(watch_accessed_variables=True, persistent=True) as tape:
                loss, self.coef, self.bias, self.predicted_train = self._backprop(weights, X_train, y_train)

            self.vars = [self.layer_reg1.weights, self.layer_reg1.bias]
            grads = tape.gradient(loss, self.vars)  
            self.optimizer.apply_gradients(zip(grads, self.vars))   
            self.calc_metrics(e, loss, y_train, self.predicted_train)
            self.batch_counter = self.batch_counter + self.batch_size

            if(e%50==0):
                print("[{0}] Loss: {1}, R2: {2}, Coef: {3}, Bias: {4}".format(e, loss, self.r2_history[e], self.coef, self.bias))


In [7]:
model = CiclicAI(epochs, lr, batch_size)
model.compile()
model.fit(data.values, data.target)
model.predict_test(data.values)

c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


NameError: name 'X_train' is not defined

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].plot([e for e in range(epochs)], model.loss_history, label="Loss")
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epochs')
ax[0].legend()

ax[1].plot([e for e in range(epochs)], model.r2_history, label="R2")
ax[1].set_ylabel('R2')
ax[1].set_xlabel('Epochs')
ax[1].legend()


fig2, ax2 = plt.subplots(2, data.values.shape[1], figsize=(20, 10))
for i in range(data.values.shape[1]):
    if data.values.shape[1] == 1:
        ax2[0].scatter(data.values[:], data.target[:], cmap='viridis')
        ax2[0].plot(data.values[:], model.predicted_train[:], color='r', zorder = 10, alpha = 0.9)
        #ax2[0].set_ylim([-300, 300])
        #ax2[0].set_xlim(-3,3)
        ax2[0].set_xlabel("Train Vs Predicted {0}".format(i))

        ax2[1].scatter(X_test[:], y_test[:], cmap='viridis')
        ax2[1].plot(X_test[:], model.predicted_test[:], color='r', zorder = 10, alpha = 0.9)
        #ax2[1].set_ylim([-200, 200])
        #ax2[1].set_xlim(-3,3)
        ax2[1].set_xlabel("Test Vs Predicted {0}".format(i))

    else:
        ax2[0, i].scatter(data.values[:, i], data.target[:], cmap='viridis')
        ax2[0, i].plot(data.values[:, i], model.predicted_train[:], color='r', zorder = 10, alpha = 0.9)
        #ax2[0, i].set_ylim([-300, 300])
        #ax2[0, i].set_xlim(-3,3)
        ax2[0, i].set_xlabel("Train Vs Predicted {0}".format(i))

        ax2[1, i].scatter(X_test[:, i], y_test[:], cmap='viridis')
        ax2[1, i].plot(X_test[:, i], model.predicted_test[:], color='r', zorder = 10, alpha = 0.9)
        #ax2[1, i].set_ylim([-200, 200])
        #ax2[1, i].set_xlim(-3,3)
        ax2[1, i].set_xlabel("Test Vs Predicted {0}".format(i))